In [5]:
import sys
sys.path.append("/home/laba/golos/whisper_fine_tuning/dataset_creation/src/")

import numpy as np
# import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm
import librosa
import glob

from utils import load_labels
from utils_model import load_model, predict_video
from utils_analysis import calculate_wer, clean_text_before_wer

In [7]:
labels = load_labels('results/filtered_labels_eval.jsonl')
model_small = load_model('small')

In [8]:
model_finetune = load_model('small', model_path="ckpt_epoch_best_epoch_iter_.pt")

In [9]:
prediction_finetune = predict_video(video_id = 72,
                                dataset_path = "eval_dataset/",
                                model = model_finetune,
                                labels = labels,
                                pred_method='base')
calculate_wer(prediction_finetune)

In [10]:
prediction_finetune.wer.mean(), prediction_finetune.wer.median()

(0.19997482020939167, 0.17857142857142858)

In [11]:
prediction_small = predict_video(video_id = 72,
                                         dataset_path = "eval_dataset/",
                                         model = model_small,
                                         labels = labels,
                                         pred_method='base')

calculate_wer(prediction_small)

In [12]:
prediction_small.wer.mean(), prediction_small.wer.median()

(0.259469253488408, 0.23076923076923078)

In [13]:
prediction_finetune

,label,prediction,clean_label,clean_prediction,wer
0,Утомилися від карантину Дарма Заплануйте втому...,втомились від карантину дарма заплануйте втоми...,утомилися від карантину дарма заплануйте втому...,втомились від карантину дарма заплануйте втоми...,0.307692
1,Слава Ісу Ви дивитесь програму Ґрати песик дуж...,слава ісу ви дивитесь програму ґрати песик дуж...,слава ісу ви дивитесь програму ґрати песик дуж...,слава ісу ви дивитесь програму ґрати песик дуж...,0.200000
2,НеКей тобто Настя Каменських записала карантин...,енкей тобто настя каменських записала карантин...,некей тобто настя каменських записала карантин...,енкей тобто настя каменських записала карантин...,0.625000
3,З точки зору бізнесу це дуже розумний хід Адже...,сточки зору бізнесу це дуже розумний хіт адже ...,з точки зору бізнесу це дуже розумний хід адже...,сточки зору бізнесу це дуже розумний хіт адже ...,0.093750
4,Вот ю гона ду вот ю гона ду Ін карантін моуд ...,what you gonna do what you gonna do in quaran...,вот ю гона ду вот ю гона ду ін карантін моуд а...,what you gonna do what you gonna do in quarant...,0.483871
...,...,...,...,...,...
126,Ага через пару років ми побачимо як Ґрета Тунб...,ага через пару років ми побачимо як ґрета тунб...,ага через пару років ми побачимо як ґрета тунб...,ага через пару років ми побачимо як ґрета тунб...,0.240000
127,миииии Не дивлячись на палання горіння та апок...,вииии не дивлячись на палання горіння та апок...,миииии не дивлячись на палання горіння та апок...,вииии не дивлячись на палання горіння та апока...,0.147059
128,Дякую Хардкісс Так подякувати гурту справді ва...,дякую хард фісс так подякувати гурту справді в...,дякую хардкісс так подякувати гурту справді ва...,дякую хард фісс так подякувати гурту справді в...,0.115385
129,Сподобалось відео Ставте лайк Не сподобалось С...,сподобалось відео ставте лайк не сподобалось с...,сподобалось відео ставте лайк не сподобалось с...,сподобалось відео ставте лайк не сподобалось с...,0.000000


In [24]:
prediction_small[prediction_small.wer>prediction_finetune.wer].shape

(90, 5)

In [25]:
(prediction_small.wer - prediction_finetune.wer).describe()

count    131.000000
mean       0.059494
std        0.122535
min       -0.500000
25%        0.000000
50%        0.058824
75%        0.121212
max        0.470588
Name: wer, dtype: float64

In [45]:
result = pd.concat([prediction_small, prediction_finetune.add_prefix('fine_tune_')], axis=1)

In [53]:
result[result.wer<result.fine_tune_wer].sort_values('fine_tune_wer', ascending=False).iloc[3]

label                         І поки ви в паніці кидаєте королю Таїланду в о...
prediction                     І поки ви у пані цекидаєте корлю Таїланду в о...
clean_label                   і поки ви в паніці кидаєте королю таїланду в о...
clean_prediction              і поки ви у пані цекидаєте корлю таїланду в ос...
wer                                                                     0.47619
fine_tune_label               І поки ви в паніці кидаєте королю Таїланду в о...
fine_tune_prediction          і поки ви у пані це кидаєте корлю та іланду в ...
fine_tune_clean_label         і поки ви в паніці кидаєте королю таїланду в о...
fine_tune_clean_prediction    і поки ви у пані це кидаєте корлю та іланду в ...
fine_tune_wer                                                           0.52381
Name: 36, dtype: object

In [52]:
result[result.wer<result.fine_tune_wer].sort_values('fine_tune_wer', ascending=False).iloc[3][['label', 'prediction', 'fine_tune_prediction']].values

array(['І поки ви в паніці кидаєте королю Таїланду в особисті повідомлення новий кліп Дуа Ліпи де вона співає що краще все ж таки вдома залишатись I would ve stayed at home Cause I was doing better alone Я закликаю вас не хвилюватись ',
       ' І поки ви у пані цекидаєте корлю Таїланду в особисі повідомлення новий кліп «Дуаліпи», де вона співає, що краще все ж таки дома залишатись. «А я закликаю вас не хвилюватись».',
       'і поки ви у пані це кидаєте корлю та іланду в особисі повідомлення новий кліп дуаліпи де вона співає що краще все ж таки дома залишатись я закликаю вас не хвилюватись '],
      dtype=object)

In [16]:
# df = pd.read_csv('results/labels_baseline_prediction.csv')
# df_eval = pd.read_csv('results/labels_eval_baseline_prediction.csv')

# df = pd.concat([df, df_eval])
# print(df.shape)
# df.head()